# MCMC - Metropolis-Hastings

In [ ]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LogNorm
import random
%matplotlib inline
from matplotlib import animation, rc
from IPython.display import HTML

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import norm

In [ ]:
fontsize = 40
scattersize=100
surf_labelpad = 50.

plt.style.use("seaborn")
plt.rcParams.update({'figure.figsize': (20,15), 'font.size': fontsize, 'axes.labelsize': fontsize, 'axes.labelpad': 15., 'text.usetex':True, 'xtick.labelsize': fontsize, 'xtick.major.pad': 20., 'ytick.labelsize': fontsize, 'ytick.major.pad': 20., })

colors = ['darkorange', 'navy', 'cornflowerblue']

In [ ]:
rng = np.random.RandomState(13)

## Example: Gaussian mixture target, Gaussian proposal

In [ ]:
def p(x):
    return 0.3*sp.stats.norm.pdf(x, loc=30, scale=10) + 0.7*sp.stats.norm.pdf(x, loc=80, scale=20)

def q(x, mu, proposal_sd):
    return sp.stats.norm.pdf(x, loc=mu, scale=proposal_sd)


In [ ]:
def plot_proposal(fig, ax, x, z, z_current, p_z_current, pdf_target, p_target, pdf_proposal, p_proposal, 
                  accepted, frame, p_proposal_z, pdf_proposal_z, prev_pdf_proposal, prev_z_current):
    ax.cla()
    if accepted:
        color_target = 'g'
    else:
        color_target = 'r'
    
    colors = ['darkorange', 'navy', 'cornflowerblue']
    
    plt.plot(x, pdf_target, linewidth=2.5, color=colors[0], label='Target pdf, $p(z)$')
    plt.plot(x, pdf_proposal, linewidth=2.5, color=colors[1], label='Proposal pdf, $q(z\,|\,z^{(\ell)})$')
    
    # Plot history of proposal:
    plt.plot(x, prev_pdf_proposal, alpha=0.2, color=colors[1], label='Proposal pdf, $q(z\,|\,z^{(\ell-1)})$')
    plt.arrow(prev_z_current, np.max(prev_pdf_proposal), z_current-prev_z_current, np.max(pdf_proposal)-np.max(prev_pdf_proposal), length_includes_head=True,
          head_width=.001, head_length=2, color=colors[1], width=0.00001)

    # Plot proposal evaluated at sample:
    plt.plot([z]*2, [0, p_proposal], color=color_target)
    plt.scatter(z,p_proposal, s=200, color=colors[1])
    
    # Plot target evaluated at sample:
    plt.plot([z]*2, [0, p_target], color=color_target)
    plt.scatter(z, p_target, s=200, color=colors[0])

    plt.title('Iteration $\ell='+str(frame)+'$', fontsize=fontsize)
    plt.xlabel('$z$')
    plt.ylabel('Probability density')
    ax.set_xlim(min(x), max(x))
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.10),
          fancybox=True, shadow=True, ncol=5, fontsize=fontsize-5)

In [ ]:
def animate(frame, fargs):
    fig = fargs['fig']
    ax = fargs['ax']
    x = fargs['x']
    p_target = fargs['hist_p_target'][frame]
    p_proposal = fargs['hist_p_proposal'][frame]
    z = fargs['trace'][frame]
    accepted = fargs['hist_accepted'][frame]
    pdf_target = fargs['pdf_target']
    pdf_proposal = fargs['hist_pdf_proposal'][frame]
    z_current = fargs['hist_z_current'][frame]
    p_z_current = fargs['hist_p_z_current'][frame]
    p_proposal_z = fargs['hist_p_proposal_z'][frame]
    pdf_proposal_z = fargs['hist_pdf_proposal_z'][frame]
    
    prev_pdf_proposal = fargs['hist_pdf_proposal'][frame-1]
    prev_z_current = fargs['hist_z_current'][frame-1]
    
    # z|X \sim N(m, P)
    ax = plot_proposal(fig, ax, x, z, z_current, p_z_current, pdf_target, p_target, pdf_proposal, p_proposal, 
                       accepted, frame, p_proposal_z, pdf_proposal_z, prev_pdf_proposal, prev_z_current)

    return ax

In [ ]:
def metropolis(x, iter=1000, z_init = 50., plot=False, proposal_sd = 30):
    if plot:
        fig, ax = plt.subplots()
        pdf_target = p(x)

    trace = []
    samples = []
    hist_p_proposal = []
    hist_pdf_proposal = []
    hist_p_target = []
    hist_accepted = []
    hist_z_current = []
    hist_p_z_current = []
    hist_pdf_proposal_z = []
    hist_p_proposal_z = []
    
    z_current = z_init
    for i in range(iter):
        # Insert your code here! Hint: You will need to assign:
        # z = ?
        # u = ?
        # alpha = ?
        # accepted = ?
        
        if plot:
            hist_pdf_proposal.append(q(x, z_current, proposal_sd))
            hist_p_proposal.append(q(z, z_current, proposal_sd))
            hist_pdf_proposal_z.append(q(x, z, proposal_sd))
            hist_p_proposal_z.append(q(z_current, z, proposal_sd))
            hist_z_current.append(z_current)
            hist_p_z_current.append(q(z_current, z_current, proposal_sd))
            hist_p_target.append(p(z))
        
        if accepted:
            z_current = z
            samples.append(z)

        trace.append(z)
        
        hist_accepted.append(accepted)
            
    if plot:
        # Generate animation from history of samples:
        print('Finished simulation. Generating animation...')
        fargs = [{'fig':fig, 'ax':ax, 'x':x, 
                                              'hist_p_target':hist_p_target, 'hist_p_proposal':hist_p_proposal,
                                              'pdf_target':pdf_target, 'hist_pdf_proposal':hist_pdf_proposal, 
                                              'trace':trace, 'hist_accepted':hist_accepted, 
                                              'hist_z_current':hist_z_current, 'hist_p_z_current':hist_p_z_current,
                                             'hist_pdf_proposal_z':hist_pdf_proposal_z, 'hist_p_proposal_z':hist_p_proposal_z}]
        ani = animation.FuncAnimation(fig, animate, frames=iter, 
                                      fargs=fargs)
        HTML(ani.to_jshtml())
        print('Finished animation. Saving...')
        ani.save('./MetropolisHastings_Gauss.gif', writer='imagemagick', fps=4, dpi=50)

    if plot:
        return samples, fargs
    else:
        return samples

In [ ]:
z_init = 0
proposal_sd = 10
x = np.linspace(-50, 150, 1000)

# 
samples, hist = metropolis(x, iter=100, z_init=z_init, proposal_sd=proposal_sd, plot=True)

In [ ]:
frame = 0
x = hist[0]['x']
p_target = hist[0]['hist_p_target'][frame]
p_proposal = hist[0]['hist_p_proposal'][frame]
z = hist[0]['trace'][frame]
accepted = hist[0]['hist_accepted'][frame]
pdf_target = hist[0]['pdf_target']
pdf_proposal = hist[0]['hist_pdf_proposal'][frame]
z_current = hist[0]['hist_z_current'][frame]
p_z_current = hist[0]['hist_p_z_current'][frame]
p_proposal_z = hist[0]['hist_p_proposal_z'][frame]
pdf_proposal_z = hist[0]['hist_pdf_proposal_z'][frame]

In [ ]:
fig, ax = plt.subplots()
plt.plot(x, pdf_target, linewidth=2.5, color=colors[0], label='Target pdf, $p(z)$')
plt.plot(x, pdf_proposal, linewidth=2.5, color=colors[1], label='Proposal pdf, $q(z\,|\,z^{(\ell)})$')

# Plot inverse proposal:
plt.plot(x, pdf_proposal_z, color=colors[2], label='$q(z\,|\,\\tilde{z})$')
plt.plot([z_current]*2, [0, p_proposal_z], color=colors[2])
plt.scatter(z_current, p_proposal_z, s=200, color=colors[2])
plt.text(z_current-30, p_proposal_z, '$q(z^{(\ell)}\,|\,\\tilde{z})$')

plt.plot([z_current]*2, [0, np.max(pdf_proposal)], color=colors[1], linestyle='dashed')
plt.plot([z]*2, [0, np.max(q(x, z, proposal_sd))], color=colors[2], linestyle='dashed')


plt.scatter(z_current, p(z_current), s=200, color=colors[0])
plt.text(z_current-25, p(z_current), '$p(z^{(\ell)})$')

# Plot proposal evaluated at sample:
plt.plot([z]*2, [0, p_proposal], color=colors[1])
plt.scatter(z,p_proposal, s=200, color=colors[1])
plt.text(z+5, p_proposal, '$q(\\tilde{z}\,|\,z^{(\ell)})$')

# Plot target evaluated at sample:
# plt.plot([z]*2, [0, p_target], color=colors[0])
plt.scatter(z, p_target, s=200, color=colors[0])
plt.text(z+5, p_target, '$p(\\tilde{z})$')

plt.legend(fontsize=fontsize, loc='upper right')
plt.xlabel('$z$')
plt.ylabel('Probability density')
ax.set_xlim(min(x), max(x))

xlims=ax.get_xlim()
ylims=ax.get_ylim()

In [ ]:
fig, ax = plt.subplots()
plt.plot(x, pdf_target, linewidth=2.5, color=colors[0], label='Target pdf, $p(z)$')

plt.legend(fontsize=fontsize, loc='upper right')
plt.xlabel('$z$')
plt.ylabel('Probability density')

ax.set_xlim(xlims)
ax.set_ylim(ylims)

In [ ]:
fig, ax = plt.subplots()
plt.plot(x, pdf_target, linewidth=2.5, color=colors[0], label='Target pdf, $p(z)$')
plt.plot(x, pdf_proposal, linewidth=2.5, color=colors[1], label='Proposal pdf, $q(z\,|\,z^{(\ell)})$')

plt.plot([z_current]*2, [0, np.max(pdf_proposal)], color=colors[1], linestyle='dashed')

plt.scatter(z_current, p(z_current), s=200, color=colors[0])
plt.text(z_current-25, p(z_current), '$p(z^{(\ell)})$')

plt.legend(fontsize=fontsize, loc='upper right')
plt.xlabel('$z$')
plt.ylabel('Probability density')

ax.set_xlim(xlims)
ax.set_ylim(ylims)

In [ ]:
fig, ax = plt.subplots()
plt.plot(x, pdf_target, linewidth=2.5, color=colors[0], label='Target pdf, $p(z)$')
plt.plot(x, pdf_proposal, linewidth=2.5, color=colors[1], label='Proposal pdf, $q(z\,|\,z^{(\ell)})$')

plt.plot([z_current]*2, [0, np.max(pdf_proposal)], color=colors[1], linestyle='dashed')

plt.scatter(z_current, p(z_current), s=200, color=colors[0])
plt.text(z_current-25, p(z_current), '$p(z^{(\ell)})$')

# Plot proposal evaluated at sample:
plt.plot([z]*2, [0, p_proposal], color=colors[1])
plt.scatter(z,p_proposal, s=200, color=colors[1])
plt.text(z+5, p_proposal, '$q(\\tilde{z}\,|\,z^{(\ell)})$')

# Plot target evaluated at sample:
plt.scatter(z, p_target, s=200, color=colors[0])
plt.text(z+5, p_target, '$p(\\tilde{z})$')

plt.legend(fontsize=fontsize, loc='upper right')
plt.xlabel('$z$')
plt.ylabel('Probability density')

ax.set_xlim(xlims)
ax.set_ylim(ylims)

In [ ]:
fig, ax = plt.subplots()
plt.plot(x, pdf_target, linewidth=2.5, color=colors[0], label='Target pdf, $p(z)$')
plt.plot(x, pdf_proposal, linewidth=2.5, color=colors[1], label='Proposal pdf, $q(z\,|\,z^{(\ell)})$')

# Plot inverse proposal:
plt.plot(x, pdf_proposal_z, color=colors[2], label='$q(z\,|\,\\tilde{z})$')
plt.plot([z_current]*2, [0, np.max(pdf_proposal)], color=colors[1], linestyle='dashed')
plt.plot([z]*2, [0, np.max(q(x, z, proposal_sd))], color=colors[2], linestyle='dashed')


plt.scatter(z_current, p(z_current), s=200, color=colors[0])
plt.text(z_current-25, p(z_current), '$p(z^{(\ell)})$')

# Plot proposal evaluated at sample:
plt.plot([z]*2, [0, p_proposal], color=colors[1])
plt.scatter(z,p_proposal, s=200, color=colors[1])
plt.text(z+5, p_proposal, '$q(\\tilde{z}\,|\,z^{(\ell)})$')

# Plot target evaluated at sample:
plt.scatter(z, p_target, s=200, color=colors[0])
plt.text(z+5, p_target, '$p(\\tilde{z})$')

plt.legend(fontsize=fontsize, loc='upper right')
plt.xlabel('$z$')
plt.ylabel('Probability density')

ax.set_xlim(xlims)
ax.set_ylim(ylims)

In [ ]:
fig, ax = plt.subplots()
plt.plot(x, pdf_target, linewidth=2.5, color=colors[0], label='Target pdf, $p(z)$')
plt.plot(x, pdf_proposal, linewidth=2.5, color=colors[1], label='Proposal pdf, $q(z\,|\,z^{(\ell)})$')

# Plot inverse proposal:
plt.plot(x, pdf_proposal_z, color=colors[2], label='$q(z\,|\,\\tilde{z})$')
plt.plot([z_current]*2, [0, p_proposal_z], color=colors[2])
plt.scatter(z_current, p_proposal_z, s=200, color=colors[2])
plt.text(z_current-30, p_proposal_z, '$q(z^{(\ell)}\,|\,\\tilde{z})$')

plt.plot([z_current]*2, [0, np.max(pdf_proposal)], color=colors[1], linestyle='dashed')
plt.plot([z]*2, [0, np.max(q(x, z, proposal_sd))], color=colors[2], linestyle='dashed')


plt.scatter(z_current, p(z_current), s=200, color=colors[0])
plt.text(z_current-25, p(z_current), '$p(z^{(\ell)})$')

# Plot proposal evaluated at sample:
plt.plot([z]*2, [0, p_proposal], color=colors[1])
plt.scatter(z,p_proposal, s=200, color=colors[1])
plt.text(z+5, p_proposal, '$q(\\tilde{z}\,|\,z^{(\ell)})$')

# Plot target evaluated at sample:
plt.scatter(z, p_target, s=200, color=colors[0])
plt.text(z+5, p_target, '$p(\\tilde{z})$')

plt.legend(fontsize=fontsize, loc='upper right')
plt.xlabel('$z$')
plt.ylabel('Probability density')

ax.set_xlim(xlims)
ax.set_ylim(ylims)

In [ ]:
samples = metropolis(x, iter=10000, z_init=z_init, proposal_sd =proposal_sd)

fig, ax = plt.subplots()
plt.plot(x, p(x), linewidth=2.5, color=colors[0], label='Target pdf, $p(z)$')
plt.hist(samples[500:], density=True, bins=50, alpha=0.5, color=colors[0])

plt.legend(fontsize=fontsize, loc='upper right')
plt.xlabel('$z$')
plt.ylabel('Probability density')

ax.set_xlim(xlims)
ax.set_ylim(ylims)